In [54]:
from flask import Flask, Response
import pickle as pkl
import logging
import pandas as pd
import sys

sys.path.append('/home/immichail/anaconda3/lib/python3.8/site-packages')

from fuzzysearch import find_near_matches


import flask
print('Flask version: ', flask.__version__)

from collections import Counter
import numpy as np
import logging
logging.getLogger().setLevel(logging.INFO)
import matplotlib.pyplot as plt
from flask import jsonify, make_response
from flask import request
import functools
import math

from werkzeug.serving import run_simple

import pymongo

from dateutil.parser import parse as parseDate
from datetime import datetime

from uuid import uuid4 as uuidf

def uuid():
    return str(uuidf())

db = pymongo.MongoClient()['ltcLongevity']

Flask version:  1.1.4


In [40]:
class NoParameterFoundException(Exception):
    pass

def req(content, param):
    if param not in content:
        raise NoParameterFoundException(param)
    return content[param]

def options_wrapper(f):
    @functools.wraps(f)
    def decorated_function(*args, **kwargs):
        if request.method == 'OPTIONS':
            return get_response('ok')
        else:
            return f(*args, **kwargs)
    return decorated_function

def token_wrapper(f):
    @functools.wraps(f)
    def decorated_function(*args, **kwargs):
        if ('token' not in request.json):
            logging.debug('No token in request')
            
            return get_response({
                    'err': 'Malformed request'
                })
        else:
            if db.users.find_one({
                'token': request.json['token']
            }) is None:
                
                logging.debug('Invalid token: %s'%request.json['token'])
                
                return get_response({
                    'err': 'Malformed request'
                })
            else:
                logging.debug('Valid token: %s'%request.json['token'])
                
                return f(*args, **kwargs)
    
    return decorated_function
        
def get_response(response):
    
    if isinstance(response, dict):
        #response = remove_nans_from_json(response)
        response = make_response(
            jsonify(response) if isinstance(response, dict) else response
        )

        response.headers['Content-Type'] = 'application/json'
        response.headers['Access-Control-Allow-Origin'] = '*'
        response.headers['Access-Control-Allow-Headers'] = '*'
        
        return response
        
    elif (isinstance(response, str)):
        response = make_response(
            response
        )
        response.headers['Access-Control-Allow-Origin'] = '*'
        response.headers['Access-Control-Allow-Headers'] = '*'
        
        return response
    
    return response

def getRecommendationsByD0():

    d0LevelGroups = ['Для ума', 'Для души', 'Для тела']

    res = [
        {
            'typeGroup': g,
            'activities': [
                {'title': i['d3LevelName'], 'd3LevelId': i['d3LevelId']} 
                for i in list(db.activities.find({'d0LevelName': g}).limit(5))
            ]

        } for g in d0LevelGroups
    ]
    
    return res

In [73]:
def fuzzySearch(q, d3LevelNames: list):
    d3LevelNamesSearch = []

    for n in d3LevelNames:
        try:
            minDist = sum([
                sorted(find_near_matches(qq.lower(), n.lower(), max_l_dist = len(qq) - 1), key = lambda x: x.dist)[0].dist
                for qq in q.split()
            ])

            d3LevelNamesSearch.append({
                'name': n, 
                'minDist': minDist
            })
        except:
            pass
        
    return d3LevelNamesSearch

def searchFilters(filters, limit = 10, offset = 0):

    filtersActivities = {}

    if ('d0LevelName' in filters)and(filters['d0LevelName'] is not None):
        filtersActivities['d0LevelName'] = {'$in': filters['d0LevelName']}

    if ('d1LevelName' in filters)and(filters['d1LevelName'] is not None):
        filtersActivities['d1LevelName'] = {'$in': filters['d1evelName']}

    if ('d2LevelName' in filters)and(filters['d2LevelName'] is not None):
        filtersActivities['d2LevelName'] = {'$in': filters['d2LevelName']}

    if ('online' in filters)and(filters['online'] is not None):
        filtersActivities['online'] = filters['online']

    if ('certificate' in filters)and(filters['certificate'] is not None):
        filtersActivities['certificate'] = filters['certificate']

    if len(filtersActivities) > 0:
        d3LevelNames = set([i['d3LevelName'] for i in db.activities.find(filtersActivities, {'d3LevelName': 1})])
    else:
        d3LevelNames = None

    if ('q' in filters)and(filters['q'] is not None):
        if d3LevelNames is None:
            d3LevelNames = set([i['d3LevelName'] for i in db.activities.find({}, {'d3LevelName': 1})])

        d3LevelNamesRelevance = fuzzySearch(filters['q'], d3LevelNames)

        d3LevelNamesRelevance = {i['name']: i['minDist'] for i in d3LevelNamesRelevance}
    else:
        d3LevelNamesRelevance = None

    filtersSchedule = {}

    if ('days' in filters)and(filters['days'] is not None):
        filtersSchedule['schedule.day'] = {'$in': filters['days']}

    if ('district' in filters)and(filters['district'] is not None):
        filtersSchedule['district'] = {'$in': filters['district']}

    if ('area' in filters)and(filters['area'] is not None):
        filtersSchedule['area'] = {'$in': filters['area']}

    if d3LevelNames is not None:
        filtersSchedule['d3LevelName'] = {'$in': list(d3LevelNames)}
        
    res = list(db.schedule.find(filtersSchedule, {'_id': False}))

    if d3LevelNamesRelevance is not None:
        res = [r for r in res if r['d3LevelName'] in d3LevelNamesRelevance]
        res = sorted(res, key = lambda x: d3LevelNamesRelevance[x['d3LevelName']])

    res = res[offset:offset + limit]  

    return res

def getCategories(param: str):
    if param in ['d0LevelName', 'd1LevelName', 'd2LevelName']:
        return list(set(db.activities.distinct(param)))
    if param in ['area', 'district']:
        return list(set(sorted([i.strip() for i in db.schedule.distinct(param) if (isinstance(i, str))and(i.strip() != '')])))
    if param in ['online', 'certificate']:
        return ['Да', "Нет"]
    if param in ['days']:
        return ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']


In [ ]:
app = Flask(__name__)
app.config['DEBUG'] = True
        
@app.route('/status', methods = ['GET', 'POST', 'OPTIONS'])
@options_wrapper
def getStatusMethod():
    content = request.json
        
    return get_response({
        'res': 'ok'
    })

@app.route('/user/set', methods = ['GET', 'POST', 'OPTIONS'])
@options_wrapper
def userSetMethod():
    content = request.json
    
    if 'uuid' not in content:
        content['uuid'] = uuid()
        db.users.insert_one(content)
    else:
        db.users.update_one({'uuid': content['uuid']}, {'$set': content})
        
    return get_response({
        'res': content['uuid']
    })

@app.route('/user/get', methods = ['GET', 'POST', 'OPTIONS'])
@options_wrapper
def userGetMethod():
    content = request.json
    
    userRecord = db.users.find_one({'uuid': content['uuid']})
    
    if userRecord is None:
        return get_response({
            'err': 'User does not exists'
        })
    else:
        del userRecord['_id']
        return get_response({
            'res': userRecord
        })
    
@app.route('/userV2/get', methods = ['GET', 'POST', 'OPTIONS'])
@options_wrapper
def userV2GetMethod():
    content = request.json
    
    name = req(content, 'name')
    surName = req(content, 'surName')
    thirdName = req(content, 'thirdName')
    dateBirth = req(content, 'dateBirth')
    dateBirth = parseDate(dateBirth)
    
    userRecord = db.usersV2.find_one({
        'name': name,
        'surName': surName,
        'thirdName': thirdName,
        'dateBirth': dateBirth
    })
    
    if userRecord is None:
        return get_response({
            'err': 'User does not exists'
        })
    else:
        del userRecord['_id']
        return get_response({
            'res': userRecord
        })

@app.route('/userV2/set', methods = ['GET', 'POST', 'OPTIONS'])
@options_wrapper
def userV2SetMethod():
    content = request.json
    
    name = req(content, 'name')
    surName = req(content, 'surName')
    thirdName = req(content, 'thirdName')
    dateBirth = req(content, 'dateBirth')
    dateBirth = parseDate(dateBirth)
    gender = req(content, 'gender')
    address = req(content, 'address')
    dateCreated = datetime.now()
    
    userId = list(db.usersV2.find().sort('userId', -1).limit(1))[0]['userId']
        
    db.usersV2.insert_one({
        'userId': userId,
        'dateCreated': dateCreated,
        'dateBirth': dateBirth,
        'address': address,
        'name': name,
        'surName': surName,
        'thirdName': thirdName,
        'gender': gender
    })
    
    return get_response({
        'res': 'ok'
    })

@app.route('/group/<groupId>', methods = ['GET', 'POST', 'OPTIONS'])
@options_wrapper
def groupGetMethod(groupId: int):
    content = request.json
    
    res = db.schedule.find_one({'groupId': int(groupId)}, {'_id': False})
    
    return get_response({
        'res': res
    })

@app.route('/work/<workId>/groupList', methods = ['GET', 'POST', 'OPTIONS'])
@options_wrapper
def workGoupListMethod(workId: int):
    content = request.json
    
    res = list(db.schedule.find({'d3LevelName': db.activities.find_one({'d3LevelId': workId})['d3LevelName']}))
    
    return get_response({
        'res': res
    })

# q = <любая строка>,
# dxLevelName = <act1>;<act2>;...<act3>
# online = Да|Нет
# certificate = Да|Нет
# days = <day1>;<day2>;...<day3>
# districts = <districts1>;<districts2>;...<districts3>
# area = <area1>;<area2>;...<area3>

def ifSplit(s):
    if s is None:
        return s
    return s.split(';')

@app.route('/search', methods = ['GET', 'POST', 'OPTIONS'])
@options_wrapper
def searchMethod():
    content = request.json
    
    limit = request.args.get('limit', default = 10, type = int)
    offset = request.args.get('offset', default = 0, type = int)
    
    q = request.args.get('q', type = str)
    
    d0LevelName = ifSplit(request.args.get('d0LevelName', type = str))
    d1LevelName = ifSplit(request.args.get('d1LevelName', type = str))
    d2LevelName = ifSplit(request.args.get('d2LevelName', type = str))
    
    online = request.args.get('online', type = str)
    if online is not None:
        online = online == 'Да'
    certificate = request.args.get('certificate', type = str)
    if certificate is not None:
        certificate = certificate == 'Да'
    
    days = ifSplit(request.args.get('days', type = str))
    district = ifSplit(request.args.get('district', type = str))
    area = ifSplit(request.args.get('area', type = str))
    
    filters = {
        'q': q,
        'd0LevelName': d0LevelName,
        'd1LevelName': d1LevelName,
        'd2LevelName': d2LevelName,
        'online': online,
        'certificate': certificate,
        'days': days,
        'district': district,
        'area': area
    }
    
    return get_response({
        'res': searchFilters(filters, limit, offset)
    })


@app.route('/startPageRecommendations', methods = ['GET', 'POST', 'OPTIONS'])
@options_wrapper
def startPageRecommendationsMethod():
    content = request.json
    
    return get_response({
        'recomendedActivities': getRecommendationsByD0()
    })

@app.route('/filter/values/<filterName>', methods = ['GET', 'POST', 'OPTIONS'])
@options_wrapper
def getFilterValuesMethod(filterName: str):
    content = request.json
      
    return get_response({
        filterName: [{'value': i} for i in getCategories(filterName)]
    })


run_simple('localhost', 11059, app)

 * Running on http://localhost:11059/ (Press CTRL+C to quit)
127.0.0.1 - - [27/May/2023 13:01:06] "OPTIONS //filter/values/d2LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:01:06] "OPTIONS //filter/values/certificate HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:01:06] "OPTIONS //filter/values/area HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:01:06] "OPTIONS //filter/values/district HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:01:06] "OPTIONS //filter/values/days HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:01:06] "OPTIONS //filter/values/online HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:01:06] "OPTIONS //filter/values/d0LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:01:06] "POST //filter/values/d2LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:01:06] "OPTIONS //filter/values/d1LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:01:06] "POST //filter/values/certificate HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:01:06] "POST //filter/values/area HTT

127.0.0.1 - - [27/May/2023 13:05:29] "OPTIONS //filter/values/d0LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:05:29] "OPTIONS //filter/values/d1LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:05:29] "POST //filter/values/d2LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:05:29] "POST //filter/values/certificate HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:05:29] "POST //filter/values/area HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:05:29] "POST //filter/values/district HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:05:29] "POST //filter/values/online HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:05:29] "POST //filter/values/days HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:05:29] "POST //filter/values/d0LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:05:29] "POST //filter/values/d1LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:05:32] "OPTIONS //startPageRecommendations?userId=94200f15-d1a5-479d-9b5a-93c525f03278 HTTP/1.0" 200 -
127.0.0.1 - -

127.0.0.1 - - [27/May/2023 13:33:08] "OPTIONS //filter/values/d0LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:33:08] "POST //filter/values/area HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:33:08] "OPTIONS //filter/values/d1LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:33:08] "POST //filter/values/days HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:33:08] "POST //filter/values/d2LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:33:08] "POST //filter/values/certificate HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:33:08] "POST //filter/values/online HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:33:08] "POST //filter/values/d0LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:33:08] "POST //filter/values/d1LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:33:10] "POST //startPageRecommendations?userId=94200f15-d1a5-479d-9b5a-93c525f03278 HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 13:33:11] "POST //filter/values/district HTTP/1.0" 200 -
127.0.0.1 - - [2

127.0.0.1 - - [27/May/2023 14:45:41] "OPTIONS //filter/values/d2LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 14:45:41] "OPTIONS //filter/values/certificate HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 14:45:41] "OPTIONS //filter/values/area HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 14:45:41] "POST //filter/values/days HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 14:45:41] "POST //filter/values/district HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 14:45:41] "POST //filter/values/online HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 14:45:41] "POST //filter/values/d0LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 14:45:41] "POST //filter/values/d1LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 14:45:41] "POST //filter/values/d2LevelName HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 14:45:41] "POST //filter/values/certificate HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 14:45:41] "POST //filter/values/area HTTP/1.0" 200 -
127.0.0.1 - - [27/May/2023 14:45:55] "OPTIONS //startPageRecomm